# Transit Boardings Report - version 2
This notebook generates two types of report:
- Standard: A single or comparison report for the overall scenario(s)
    - This includes sub-mode daily boardings and TOD totals (AM, MD, PM, NT) 
- Detailed: A detailed report for selected links.
    - This includes Daily and TOD boardings for user specified route(s)


### This is a work-in-progress and currently (9/24/2021) ONLY to be used by a qualified developer!

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly import tools

In [ ]:
# One or two scenarios?
# 
# This variable ('scenarios') was called 'bases' in the first version of this notebook.
#
scenarios = {'Base Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'
             #,'Comparative Model':r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
            } 

In [ ]:
# Reference data: CSV file containing list of _ALL_ transit routes:
all_transit_routes_csv_fn = \
r'G:\Regional_Modeling\1A_Archives\LRTP_2018\2016 Scen 00_08March2019_MoDXoutputs\Databases\Statewide_Routes_2018S.csv'

#### User input required: supply name of CSV file with list of routes on which to report.

In [ ]:
# Two types of reports:
# Standard (all routes in the input CSV file you suppy) or detailed (in-line list of routes.)

# CSV file containing list of transit routes for which to generate this report:
routes_csv_fn = r'G:/Data_Resources/DataStore/transit info.csv'

# Read this CSV file into a pandas dataframe:
routes_df = pd.read_csv(routes_csv_fn)

# Specify list of routes for which to generate report.
# By default this is all routes in the 'routefile' dataframe:
route_list = routes_df['Route_ID']

In [ ]:
scenario = scenarios['Base Model']
base = scenario + r'out/'
tod = 'AM'
x = tod + '/'
# Get fully-qualfied paths to all CSVs for the 'AM' period
fq_csv_fns = glob.glob(os.path.join(base,x,r'*.csv'))

In [ ]:
fq_csv_fns

In [ ]:
temp_df0 = pd.read_csv(fq_csv_fns[0])

In [ ]:
len(temp_df0)

In [ ]:
temp_df0 = temp_df0[temp_df0['ROUTE'].isin(route_list)]

In [ ]:
len(temp_df0)

In [ ]:
temp_df0.set_index(['ROUTE','STOP'])

In [ ]:
temp_df20 = pd.read_csv(fq_csv_fns[20])

In [ ]:
len(temp_df20)

In [ ]:
temp_df20 = temp_df20[temp_df20['ROUTE'].isin(route_list)]

In [ ]:
len(temp_df20)

In [ ]:
temp_df20.set_index(['ROUTE','STOP'])

In [ ]:
tablist = [temp_df0, temp_df20]

In [ ]:
blah = reduce(lambda a, b: a.add(b, fill_value=0), tablist)

In [ ]:
blah

In [ ]:
# Import transit assignment result CSV files.
#
# Stackoverflow article used by Margaret as reference for summing the data frames:
# https://stackoverflow.com/questions/11106823/adding-two-pandas-dataframes
# 
def import_transit_assignment(scenario):
    '''bring in data and combine into sum tables for daily and put into a dictionary'''
    base = scenario + r'out/'
    tods = ["AM", "MD", "PM", "NT"]
    # At the end of execution of this function, the dictionary variable'TODsums' will contain all the TOD summed results:
    # one key-value-pair for each 'tod' and one for the daily total ('daily').
    # 'TODsums' is the return value of this function.
    TODsums = { 'AM' : None, 'MD' : None, 'PM' : None, 'NT' : None }

    # Import CSV files and create sum tables for each TOD and for the day as a whole
    for tod in tods:
        # Get full paths to _all_ CSV files for the current t-o-d (a.k.a. 'time period')
        x = tod + '/' 
        fq_csv_fns = glob.glob(os.path.join(base,x,r'*.csv'))
        # 'tablist' : List of all the dataframes created from reading in the all the CSV files for the current t-o-d
        tablist = []
        for csv_file in fq_csv_fns:
            # Read CSV file into dataframe, set indices, and append to 'tablist'
            tablist.append(pd.read_csv(csv_file).set_index(['ROUTE','STOP']))
        #
        # Filter dataframe to include rows where 'ROUTE' is one of those selected to report on
        # BK question: Why wasn't this done earlier, before the dataframe was added to 'tablist'?
        for t in range(len(tablist)):
            tablist[t] = tablist[t][tablist[t].index.get_level_values('ROUTE').isin(route_list)]
        #
        
        # Sum the tables for the current TOD
        TODsums[tod] = reduce(lambda a, b: a.add(b, fill_value=0), tablist)
    # end_for over all tod's
    
    # Sum of all the TOD sum tables into a single sum for the _entire_ day.
    # BUT this isn't a simple sum, as the data frames can - and do - have different lengths.
    # We first have to join (pandas: 'merge') them...
    
    j1 = pd.merge(TODsums['AM'], TODsums['MD'], on=['ROUTE', 'STOP'], how='outer')
    j2 = pd.merge(j1, TODsums['PM'], on=['ROUTE', 'STOP'], how='outer')
    j3 = pd.merge(j2, TODsums['NT'], on=['ROUTE', 'STOP'], how='outer')
    
    # The following code will not work - commented out for now:
    #
    # Note: This is where the 'daily' key is added to TODsums.
    # TODsums['daily'] = reduce(lambda a, b: a.add(b, fill_value=0), j)
    
    # Temp hack for now, to make merge results avilable for inspection outside of this function.
    TODsums['daily'] = j3
    
    # Ensure that the ROUTE and STOP columns aren't indices
    for x in TODsums.keys():
        TODsums[x] = TODsums[x].reset_index()
    return TODsums
#

In [ ]:
results = import_transit_assignment(scenarios['Base Model'])

In [ ]:
results['daily']

In [ ]:
df1 = pd.DataFrame([(1, 2, 1000),(3, 4, 2000),(5, 6, 3000)], columns=['a','b', 'ix'])
df1.set_index('ix')

In [ ]:
df2 = pd.DataFrame([(100,200, 2000),(300,400, 1000),(500,600, 4000), (700, 800, 5000)], columns=['a','b', 'ix'])
df2.set_index('ix')

In [ ]:
df_add = df1.add(df2, fill_value=0)

In [ ]:
df_add

In [ ]:
import datetime as dt

In [ ]:
prices = pd.DataFrame([[553.0, 555.5, 549.3, 554.11, 0],
                       [556.8, 556.8, 544.05, 545.92, 545.92],
                       [545.5, 546.89, 540.97, 542.04, 542.04]],
                       index=[dt.datetime(2014,11,4), dt.datetime(2014,11,5), dt.datetime(2014,11,6)],
                       columns=['Open', 'High', 'Low', 'Close', 'Adj Close'])

In [ ]:
prices

In [ ]:
corrections = pd.DataFrame([[0, 555.22], [1238900, 0]],
                    index=[dt.datetime(2014,11,3), dt.datetime(2014,11,4)],
                    columns=['Volume', 'Adj Close'])

In [ ]:
corrections

In [ ]:
dates = pd.DataFrame(prices.index, columns = ['Dates']).append(pd.DataFrame(corrections.index, columns = ['Dates'])).drop_duplicates('Dates').set_index('Dates').sort_values('Dates')

In [ ]:
dates

In [ ]:
df_corrections = dates.join(corrections).fillna(0)

In [ ]:
df_corrections

In [ ]:
df_prices = dates.join(prices).fillna(0)

In [ ]:
df_prices

In [ ]:
for col in prices.columns:
    if col in corrections.columns:
        df_prices[col]+=df_corrections[col]
        del df_corrections[col]

In [ ]:
df_prices

In [ ]:
df_corrections

In [ ]:
df_prices = df_prices.join(df_corrections)

In [ ]:
df_prices

In [ ]:
# results = import_transit_assignment(scenarios['Base Model'])

In [ ]:
# results['NT']

In [ ]:
# Define data structure and function to map a TransCAD 'Mode' to the corresponding 'Meta-mode'
_mode_to_metamode_mapping_table = {
    1:  'MBTA_Bus',
    2:  'MBTA_Bus',
    3:  'MBTA_Bus' ,
    4:  'Light_Rail',
    5:  'Heavy_Rail',
    6:  'Heavy_Rail',
    7:  'Heavy_Rail',
    8:  'Heavy_Rail',
    9:  'Commuter_Rail',
    10: 'Ferry',
    11: 'Ferry',
    12: 'Light_Rail',
    13: 'Light_Rail',
    14: 'Shuttle_Express',
    15: 'Shuttle_Express',
    16: 'Shuttle_Express',
    17: 'RTA',
    18: 'RTA',
    19: 'RTA',
    20: 'RTA',
    21: 'RTA',
    22: 'RTA',
    23: 'Private',
    24: 'Private',
    25: 'Private',
    26: 'Private',
    27: 'Private',
    28: 'Private',
    29: 'Private',
    30: 'Private',
    31: 'Private',
    32: 'Commuter_Rail',
    33: 'Commuter_Rail',
    34: 'Commuter_Rail',
    35: 'Commuter_Rail',
    36: 'Commuter_Rail',
    37: 'Commuter_Rail',
    38: 'Commuter_Rail',
    39: 'Commuter_Rail',
    40: 'Commuter_Rail',
    41: 'Commuter_Rail',
    42: 'Commuter_Rail',
    43: 'Commuter_Rail',
    44: 'Commuter_Rail',
    70: 'Walk' }

def mode_to_metamode(mode):
    retval = 'None'
    if mode in _mode_to_metamode_mapping_table:
        return _mode_to_metamode_mapping_table[mode]
    # end_if
    return retval

In [ ]:
def set_up_metamode_table(scenario):
    '''flag each route type by metaMode'''
    routemode = pd.read_csv(scenario + r'Databases/Statewide_Routes_2018S.csv', 
                            usecols=["Routes_ID", "Mode"]).drop_duplicates()
    routemode['metaMode'] = routemode.apply(lambda x: mode_to_metamode(x['Mode']), axis=1)
    return routemode

In [ ]:
def join_and_agg(TODSums, routemode):
    '''aggregate the on and offs by route or metaMode'''
#set the group by field depending on if standard or detailed report
    if len(route_list) > 0:
        agg = 'ROUTE'
    else: 
        agg = 'metaMode'

    for x in TODsums.keys():
        if len(routeList)> 0:
            TODsums[x] = routefile.merge(TODsums[x], how='outer', left_on='Route_ID', right_on='ROUTE')
            TODsums[x]['ROUTE'] = TODsums[x]['Route_Name'].str.split('.:()').str[0]
        #join each table to route mode
            TODsums[x] = routemode.merge(TODsums[x], how='right', left_on='Routes_ID', right_on='Route_ID')
        else:
            TODsums[x] = routemode.merge(TODsums[x], how='right', left_on='Routes_ID', right_on='ROUTE')
        #sum all On/Off fields by metamode 
        TODsums[x] = TODsums[x].groupby([agg])[['DirectTransferOff','DirectTransferOn','DriveAccessOn','EgressOff','Off','On',
                                                'WalkAccessOn','WalkTransferOff','WalkTransferOn']].agg('sum').reset_index()
    return TODsums

In [ ]:
def plots(scen2, g):
    '''make graphs!'''
    onfdict = {}
    if len(routeList) > 0: #if detailed/standard use appropriate agg field to graph
        xVal = 'ROUTE'
    else:
        xVal = 'metaMode'
    # Make faceted graph for base and comparative scenario together    
    if 'Comparative Model' in scenarios.keys():
        scen2['compGraph']={}
        for tod in scen['Base Model'].keys(): #add flag field so can smush both scenario tables into one
            scen2['Base Model'][tod]['Scenario']='Base'
            scen2['Comparative Model'][tod]['Scenario']='Comparative'
            scen2['compGraph'][tod]=scen2['Base Model'][tod].append(scen2['Comparative Model'][tod]) #smoosh
            
        TODsums = scen2['compGraph']
        
        for z in TODsums.keys(): #make graphs (stacked bar)
            #set up table so can use for facets (wide to long format and flag field)
            lng = TODsums[z].drop(['DirectTransferOff','EgressOff','Off','On','WalkTransferOff'], axis = 1).melt(id_vars = [xVal, 'Scenario'], value_name = 'Count', ignore_index=False) #long to allow flag
            #lng=lng.reset_index() #Scenario will be facet field
            #make sure ids are strings for graphing purposes
            lng[xVal] = lng[xVal].astype(str)
            #make faceted stacked bar graphs (on and off dif graphs)
            on_off = px.bar(lng, x = xVal, y = 'Count', color = 'variable', facet_col = 'Scenario',title='Base and Comparative Model: '+z+' Boardings')
            #save graphs
            onfdict[z] = on_off
    else: #if only BASE
        TODsums = scen2['Base Model']
        for z in TODsums.keys(): #go through TOD
            TODsums[z][xVal] = TODsums[z][xVal].astype(str) #make safe for graphing
            onfdict[z] = px.bar(TODsums[z], x=xVal, y=['DirectTransferOn','DriveAccessOn','WalkAccessOn','WalkTransferOn'],  
                               title='Base Model '+z+' Boardings') #graph!
    return onfdict

In [ ]:
def diftab(scen):
#make difference tables
    if len(routeList) > 0: #if detailed/standard use appropriate agg field to graph
        xVal = 'ROUTE'
    else:
        xVal = 'metaMode'
    if len(scenarios.keys()) ==  2:#if two scenarios
        for z in TODsums.keys(): #for each TOD
            #take the difference (and replace for TOD in the global TODsums)
            TODsums[z] = (scen['Base Model'][z].set_index(xVal).drop('Scenario', axis=1) - scen['Comparative Model'][z].set_index(xVal).drop('Scenario', axis=1)).reset_index()
            #make sure ids are strings for graphing purposes
            TODsums[z][xVal] = TODsums[z][xVal].astype(str)
            onfdict[z] = px.bar(TODsums[z], x=xVal, y=['DirectTransferOn','DriveAccessOn','WalkAccessOn','WalkTransferOn'],  
                               title='Difference in '+z+' Boardings')
    scen['Difference'] = [TODsums, onfdict] #add difference data and graphs to scen dict
    return scen

In [ ]:
# call things
# write a SUPER FUNCTION!!! (which calls all functions)

# 'scen' is a two-level dict in which full set of results are accumulated.
# Level 1 = scenario
# Level 2 = tod
scen = {}

for g in scenarios.keys(): #run all these functions for each scenario
    TODsums = import_transit_assignment(scenarios[g]) #get the total boarding per route per TOD
    # Have to generate a route-to-mode=-to-metamode amapping table for _each scenario_
    # because the list of routes MAY NOT be the same for each scenario!
    routemode = set_up_metamode_table(scenarios[g]) 
    TODsums = join_and_agg(TODsums, routemode) #aggregate by mode or route
    scen[g] = TODsums
    #make graphs
scen['compGraph'] = plots(scen,g)  #package the data for showing graphs

#this is just for getting the difference to happen
if len(scenarios.keys())==2:
    scen = diftab(scen)


## Look at Results by TOD

In [ ]:
#Show AM Boardings
scen['compGraph']['AM'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(scenarios.keys())==2:
    scen['Difference'][1]['AM'].show()


In [ ]:
#Show MD Boardings
scen['compGraph']['MD'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(scenarios.keys())==2:
    scen['Difference'][1]['MD'].show()

In [ ]:
#Show PM Boardings
scen['compGraph']['PM'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(scenarios.keys())==2:
    scen['Difference'][1]['PM'].show()

In [ ]:
#Show NT Boardings
scen['compGraph']['NT'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(scenarios.keys())==2:
    scen['Difference'][1]['NT'].show()

In [ ]:
#Show Daily Boardings
scen['compGraph']['daily'].show()
#if comparative, also show graphs of boarding differences (base - comparative) 
if len(scenarios.keys())==2:
    scen['Difference'][1]['daily'].show()